Web Scrapping for NSE Academy   # most data in Brochure

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re

# -------------------- DRIVER SETUP --------------------
def get_driver():
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--window-size=1280,720")
    options.add_argument("--log-level=3")
    # options.add_argument("--headless=new")  # Uncomment for silent scraping
    driver = uc.Chrome(options=options)
    return driver

# -------------------- CLEAN TEXT --------------------
def clean_text(text):
    return " ".join(text.split()) if text else text

# -------------------- EXTRACT FEE FROM HTML --------------------
def extract_fee_from_html(soup):
    """
    Extract fee information from HTML content, specifically looking for
    the pattern provided in the example.
    """
    try:
        # Look for paragraph containing fee information
        fee_patterns = [
            r"course.*fee.*\d+",
            r"fee.*\d+",
            r"₹.*\d+",
            r"rs\.?.*\d+",
            r"rupees.*\d+"
        ]
        
        # Check all paragraphs for fee information
        for p in soup.find_all('p'):
            text = p.get_text().lower()
            if any(re.search(pattern, text) for pattern in fee_patterns):
                # Check if this paragraph contains strong tag with fee info
                strong_tag = p.find('strong')
                if strong_tag:
                    return clean_text(p.get_text())
        
        # Alternative approach: look for specific HTML structure
        for p in soup.find_all('p'):
            strong_tag = p.find('strong')
            if strong_tag and 'fee' in strong_tag.get_text().lower():
                return clean_text(p.get_text())
                
    except Exception as e:
        print(f"⚠️ Fee extraction from HTML issue: {e}")
    
    return "Not Found"

# -------------------- SCRAPER --------------------
def scrape_course_data(url):
    driver = get_driver()
    try:
        print(f"🌐 Accessing URL: {url}")
        driver.get(url)
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(3)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # ---------- Course Name ----------
        course_name = "Not Found"
        course_name_selectors = [
            "h1.page-title.text-uppercase",
            "div.page-header h1",
            "h1.course-title",
            "h1.title",
            "h1",
            "title"
        ]
        for selector in course_name_selectors:
            element = soup.select_one(selector)
            if element:
                course_name = clean_text(element.get_text(strip=True))
                break
        print(f"📛 Course Name: {course_name}")

        # ---------- About Course ----------
        about_course = "Not Found"
        try:
            active_tab = soup.find("div", {"class": "tab-pane active show"})
            if not active_tab:
                tab_panes = soup.find_all("div", class_="tab-pane")
                if tab_panes:
                    active_tab = tab_panes[0]  # fallback

            if active_tab:
                parts = []
                for tag in active_tab.find_all(["p", "li", "ul", "ol"]):
                    if tag.name == "li":
                        text = clean_text(tag.get_text(" ", strip=True))
                        parts.append(f"- {text}")
                    elif tag.name == "p":
                        text = clean_text(tag.get_text(" ", strip=True))
                        if text:
                            parts.append(text)
                    elif tag.name in ["ul", "ol"]:
                        for li in tag.find_all("li"):
                            text = clean_text(li.get_text(" ", strip=True))
                            parts.append(f"- {text}")
                if parts:
                    about_course = "\n".join(parts)
        except Exception as e:
            print(f"⚠️ About course extraction issue: {e}")

        print(f"📝 About Course: {about_course[:100]}..." if about_course else "📝 About Course: Not Found")

        # ---------- Syllabus ----------
        syllabus = "Not Found"
        accordion = soup.find("div", class_="course-sc-syllabus")
        if accordion:
            modules = []
            for card in accordion.find_all("div", class_="card"):
                header = card.find("button")
                module_title = header.get_text(strip=True) if header else "Untitled Module"

                content_div = card.find("div", class_="course-sc-syllabus-content")
                lessons = []
                if content_div:
                    for li in content_div.find_all("li"):
                        lessons.append("- " + li.get_text(strip=True))
                    for p in content_div.find_all("p"):
                        lessons.append("- " + p.get_text(" ", strip=True))

                if lessons:
                    modules.append(f"{module_title}\n" + "\n".join(lessons))
                else:
                    modules.append(module_title)

            if modules:
                syllabus = "\n\n".join(modules)

        print(f"📚 Syllabus extracted ({len(syllabus.splitlines())} lines)")

        # ---------- Extract fields from table ----------
        def extract_table_field(header_text):
            try:
                tables = soup.find_all("table")
                for table in tables:
                    headers = [h.get_text(strip=True).lower() for h in table.find_all("th")]
                    for row in table.find_all("tr"):
                        cells = row.find_all("td")
                        if len(cells) == len(headers):
                            for i, header in enumerate(headers):
                                if header_text.lower() in header:
                                    return clean_text(cells[i].get_text(strip=True))
                return "Not Found"
            except Exception as e:
                print(f"⚠️ Error extracting {header_text}: {e}")
                return "Not Found"

        duration = extract_table_field("Total Duration")
        learning_mode = extract_table_field("Course Type")
        level = extract_table_field("level") or "Not Found"
        language = extract_table_field("language") or "Not Found"
        
        # Extract price from table first, then try HTML extraction
        price = extract_table_field("price") or extract_table_field("fee") or "Not Found"
        if price == "Not Found":
            price = extract_fee_from_html(soup)
            
        certificate = extract_table_field("certificate") or extract_table_field("Certification") or "Not Found"

        print(f"⏳ Duration: {duration}")
        print(f"📊 Course Level: {level}")
        print(f"🗣️ Language: {language}")
        print(f"🎓 Learning Mode: {learning_mode}")
        print(f"💰 Price: {price}")
        print(f"🏆 Certificate: {certificate}")

        # ---------- Brochure ----------
        brochure_link = "Not Found"
        try:
            brochure_btn = soup.find("a", href=lambda href: href and (".pdf" in href.lower() or "brochure" in href.lower()))
            if brochure_btn and brochure_btn.get("href"):
                brochure_link = brochure_btn["href"]
                if not brochure_link.startswith("http"):
                    base_url = "/".join(url.split("/")[:3])
                    brochure_link = base_url + brochure_link
        except Exception as e:
            print(f"⚠️ Brochure extraction issue: {e}")
        print(f"📄 Brochure: {brochure_link}")

        # ---------- Eligibility ----------
        eligibility_parts = []
        try:
            for box in soup.find_all("div", class_="cs-textbox"):
                title_div = box.find("div", class_="cs-textbox-title")
                info_div = box.find("div", class_="cs-textbox-info")
                if title_div and "eligibility" in title_div.get_text(strip=True).lower() and info_div:
                    list_items = info_div.find_all("li")
                    if list_items:
                        eligibility_parts.append(
                            "\n".join([f"- {li.get_text(strip=True)}" for li in list_items])
                        )
                    else:
                        eligibility_parts.append(clean_text(info_div.get_text(" ", strip=True)))
                    break

            section = soup.find("div", class_="course-section-content")
            if section:
                ul = section.find("ul")
                if ul:
                    lis = ul.find_all("li")
                    if lis:
                        eligibility_parts.append(
                            "\n".join([f"- {li.get_text(strip=True)}" for li in lis])
                        )
                    else:
                        eligibility_parts.append(clean_text(section.get_text(" ", strip=True)))
        except Exception as e:
            print(f"⚠️ Eligibility extraction issue: {e}")

        eligibility = "\n".join([part for part in eligibility_parts if part]) or "Not Found"
        print(f"👥 Eligibility:\n{eligibility}")

        return [
            course_name, about_course, syllabus, duration, level, language,
            learning_mode, price, certificate, eligibility, brochure_link, url
        ]

    except Exception as e:
        print(f"🔥 Scraping failed: {str(e)}")
        import traceback
        traceback.print_exc()
        return ["Error"] * 12
    finally:
        driver.quit()
        print("🚪 Browser closed")

# -------------------- SAVE TO EXCEL --------------------
def save_to_excel(data, file_path):
    columns = [
        "Course Name", "About Course", "Syllabus", "Duration", "Course Level",
        "Language", "Learning Mode", "Price", "Certificate", "Eligibility", "Brochure", "Course URL"
    ]
    try:
        if os.path.exists(file_path):
            df = pd.read_excel(file_path)
            for col in columns:
                if col not in df.columns:
                    df[col] = None
        else:
            df = pd.DataFrame(columns=columns)

        if data[-1] in df["Course URL"].values:
            print(f"🔄 Course already exists: {data[0]}")
            return False

        new_row = pd.DataFrame([dict(zip(columns, data))])
        df = pd.concat([df, new_row], ignore_index=True)
        df.to_excel(file_path, index=False)
        print(f"💾 Saved data for: {data[0]}")
        return True

    except Exception as e:
        print(f"❌ Excel save error: {e}")
        return False

# -------------------- READ URLS FROM EXCEL --------------------
def read_urls_from_excel(file_path, sheet_name=0, url_column="Course URL"):
    """
    Read URLs from an Excel file
    
    Parameters:
    file_path (str): Path to the Excel file
    sheet_name (str/int): Name or index of the sheet to read from
    url_column (str): Name of the column containing URLs
    
    Returns:
    list: List of URLs to scrape
    """
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        # Check if the URL column exists
        if url_column not in df.columns:
            print(f"❌ Column '{url_column}' not found in the Excel file.")
            print(f"Available columns: {list(df.columns)}")
            return []
        
        # Filter out empty URLs and return the list
        urls = df[url_column].dropna().tolist()
        print(f"📖 Found {len(urls)} URLs in the Excel file")
        return urls
        
    except Exception as e:
        print(f"❌ Error reading Excel file: {e}")
        return []

# -------------------- MAIN EXECUTION --------------------
if __name__ == "__main__":
    # Define paths
    input_file_path = r"C:\Users\taslim.siddiqui\Downloads\NSE Acadeny input data.xlsx"  # Update this path
    output_file_path = r"C:\Users\taslim.siddiqui\Downloads\NSE_Scraped.xlsx"  # Update this path
    
    # Read URLs from Excel file
    course_urls = read_urls_from_excel(input_file_path)
    
   
    
    print("🚀 Starting scraping process...")
    
    # Scrape each URL
    for course_url in course_urls:
        print(f"\n📖 Processing: {course_url}")
        course_data = scrape_course_data(course_url)
        if all(item != "Error" for item in course_data):
            save_to_excel(course_data, output_file_path)
        else:
            print(f"❌ Failed to scrape complete data for {course_url}")
    
    print("✅ Process completed")

📖 Found 41 URLs in the Excel file
🚀 Starting scraping process...

📖 Processing: https://www.nseacademy.info/aicpa/#courseListing
🌐 Accessing URL: https://www.nseacademy.info/aicpa/#courseListing
📛 Course Name: AICPA & CIMA
📝 About Course: Not Found...
📚 Syllabus extracted (1 lines)
⏳ Duration: Not Found
📊 Course Level: Not Found
🗣️ Language: Not Found
🎓 Learning Mode: Not Found
💰 Price: Not Found
🏆 Certificate: Not Found
📄 Brochure: Not Found
👥 Eligibility:
Not Found
🚪 Browser closed
🔄 Course already exists: AICPA & CIMA

📖 Processing: https://www.nseacademy.info/aicpa/index.html#faq
🌐 Accessing URL: https://www.nseacademy.info/aicpa/index.html#faq
📛 Course Name: AICPA & CIMA
📝 About Course: Not Found...
📚 Syllabus extracted (1 lines)
⏳ Duration: Not Found
📊 Course Level: Not Found
🗣️ Language: Not Found
🎓 Learning Mode: Not Found
💰 Price: Not Found
🏆 Certificate: Not Found
📄 Brochure: Not Found
👥 Eligibility:
Not Found
🚪 Browser closed
🔄 Course already exists: AICPA & CIMA

📖 Processi

Traceback (most recent call last):
  File "C:\Users\taslim.siddiqui\AppData\Local\Temp\ipykernel_11360\3950342964.py", line 67, in scrape_course_data
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
  File "c:\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 129, in until
    value = method(self._driver)
            ^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\selenium\webdriver\support\expected_conditions.py", line 104, in _predicate
    return driver.find_element(*locator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 922, in find_element
    return self.execute(Command.FIND_ELEMENT, {"using": by, "value": value})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 454, in execute
    self.error_handler.check_re

🚪 Browser closed
❌ Failed to scrape complete data for https://www.nseacademy.info/aicpa/index.html#coursess

📖 Processing: https://www.nseindia.com/learn/data-analytics-using-r
🌐 Accessing URL: https://www.nseindia.com/learn/data-analytics-using-r
📛 Course Name: Data Analytics using R
📝 About Course: Organizations see data as an asset. Managed well, data can help define long-term business strategy, ...
📚 Syllabus extracted (1 lines)
⏳ Duration: 25 Hours
📊 Course Level: Not Found
🗣️ Language: Not Found
🎓 Learning Mode: Online Self-Paced
💰 Price: Not Found
🏆 Certificate: Not Found
📄 Brochure: https://nsearchives.nseindia.com/web/sites/default/files/2024-05/GT Data Analytics using R  1.pdf
👥 Eligibility:
Not Found
🚪 Browser closed
🔄 Course already exists: Data Analytics using R

📖 Processing: https://www.nseindia.com/learn/online-recorded-courses-derivatives-trading-strategies
🌐 Accessing URL: https://www.nseindia.com/learn/online-recorded-courses-derivatives-trading-strategies
📛 Course N

Traceback (most recent call last):
  File "C:\Users\taslim.siddiqui\AppData\Local\Temp\ipykernel_11360\3950342964.py", line 66, in scrape_course_data
    driver.get(url)
  File "c:\Python311\Lib\site-packages\undetected_chromedriver\__init__.py", line 665, in get
    return super().get(url)
           ^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 479, in get
    self.execute(Command.GET, {"url": url})
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 454, in execute
    self.error_handler.check_response(response)
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 232, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=139.0.7258.139)
Stacktrace:
	GetHandleVerifier [0x0x53ffc

🚪 Browser closed
❌ Failed to scrape complete data for https://www.nseindia.com/learn/certifications-live-online-advanced-equity-research-technical-analysis

📖 Processing: https://www.nseindia.com/learn/investment-strategies-for-financial-markets
🌐 Accessing URL: https://www.nseindia.com/learn/investment-strategies-for-financial-markets
🔥 Scraping failed: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=139.0.7258.139)
Stacktrace:
	GetHandleVerifier [0x0x95ffc3+65331]
	GetHandleVerifier [0x0x960004+65396]
	(No symbol) [0x0x753f63]
	(No symbol) [0x0x732e69]
	(No symbol) [0x0x7c7c7e]
	(No symbol) [0x0x7e24d9]
	(No symbol) [0x0x7c12d6]
	(No symbol) [0x0x790910]
	(No symbol) [0x0x791784]
	GetHandleVerifier [0x0xba38b3+2439203]
	GetHandleVerifier [0x0xb9eae2+2419282]
	GetHandleVerifier [0x0x98712a+225434]
	GetHandleVerifier [0x0x976e08+159096]
	GetHandleVerifier [0x0x97dd5d+187597]
	GetHandleVerifier [0x0x967ad8+96840]
	GetH

...

🚪 Browser closed
❌ Failed to scrape complete data for https://www.nseindia.com/learn/self-study-ncfm-modules-foundation-mutual-funds

📖 Processing: https://www.nseindia.com/learn/self-study-ncfm-modules-foundation-securities-market
🌐 Accessing URL: https://www.nseindia.com/learn/self-study-ncfm-modules-foundation-securities-market
🔥 Scraping failed: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=139.0.7258.139)
Stacktrace:
	GetHandleVerifier [0x0x81ffc3+65331]
	GetHandleVerifier [0x0x820004+65396]
	(No symbol) [0x0x613f63]
	(No symbol) [0x0x5f2e69]
	(No symbol) [0x0x687c7e]
	(No symbol) [0x0x6a24d9]
	(No symbol) [0x0x6812d6]
	(No symbol) [0x0x650910]
	(No symbol) [0x0x651784]
	GetHandleVerifier [0x0xa638b3+2439203]
	GetHandleVerifier [0x0xa5eae2+2419282]
	GetHandleVerifier [0x0x84712a+225434]
	GetHandleVerifier [0x0x836e08+159096]
	GetHandleVerifier [0x0x83dd5d+187597]
	GetHandleVerifier [0x0x827ad8+96840]
	GetHandle

Traceback (most recent call last):
  File "C:\Users\taslim.siddiqui\AppData\Local\Temp\ipykernel_11360\3950342964.py", line 66, in scrape_course_data
    driver.get(url)
  File "c:\Python311\Lib\site-packages\undetected_chromedriver\__init__.py", line 665, in get
    return super().get(url)
           ^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 479, in get
    self.execute(Command.GET, {"url": url})
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 454, in execute
    self.error_handler.check_response(response)
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 232, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=139.0.7258.139)
Stacktrace:
	GetHandleVerifier [0x0x81ffc

🚪 Browser closed
❌ Failed to scrape complete data for https://www.nseindia.com/learn/self-study-ncfm-modules-foundation-securities-market

📖 Processing: https://www.nseindia.com/learn/sbi-certification-courses
🌐 Accessing URL: https://www.nseindia.com/learn/sbi-certification-courses
📛 Course Name: Banking Courses by SBI
📝 About Course: SBI courses are crafted by SBI learning and development team, ensuring that participants will receiv...
📚 Syllabus extracted (1 lines)
⏳ Duration: Not Found
📊 Course Level: Not Found
🗣️ Language: Not Found
🎓 Learning Mode: Online
💰 Price: Not Found
🏆 Certificate: Not Found
📄 Brochure: https://nsearchives.nseindia.com/web/sites/default/files/2025-01/Brochure_SBI_141223.pdf
👥 Eligibility:
Not Found
🚪 Browser closed
💾 Saved data for: Banking Courses by SBI

📖 Processing: https://www.nseindia.com/learn/cyber-security-fundamentals
🌐 Accessing URL: https://www.nseindia.com/learn/cyber-security-fundamentals
📛 Course Name: Cyber Security Fundamentals
📝 About Cou

Traceback (most recent call last):
  File "C:\Users\taslim.siddiqui\AppData\Local\Temp\ipykernel_11360\3950342964.py", line 67, in scrape_course_data
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
  File "c:\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 129, in until
    value = method(self._driver)
            ^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\selenium\webdriver\support\expected_conditions.py", line 104, in _predicate
    return driver.find_element(*locator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 922, in find_element
    return self.execute(Command.FIND_ELEMENT, {"using": by, "value": value})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 454, in execute
    self.error_handler.check_re

🚪 Browser closed
❌ Failed to scrape complete data for https://www.nseindia.com/learn/online-recorded-courses-global-markets-investments
✅ Process completed
